## Author: Wonho Lim 
#### E-mail: wonholim02@gmail.com
#### Environment: Python 3.10.4 (ipykernel)
#### Last Update: 2022-06-01
#### Description: Web crawling code for collecting Thailand data using SET website.

### Utilized BS4 and Selenium Web Driver for Crawling - Visualized by Pandas

In [1]:
import bs4
import selenium
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.select import Select
from selenium.webdriver.common.by import By
import csv 
import pandas as pd 
from platform import python_version
import requests
import lxml 

In [2]:
print(python_version())

3.10.4


In [3]:
print(pd.__version__)

1.4.2


### Crawled Website
Stock Exchange of Thailand(SET) MAIN: https://classic.set.or.th/en/products/equities/equities_p1.html /
SET Equities: https://classic.set.or.th/set/companyprofile.do?symbol=2S&ssoPageId=4&language=en&country=US

### Basic setting for csv file that will store 재무정보 data - Order of rows are based on 재무정보 데이터소스

In [4]:
cover = open('Thailand_2022_SET_2022-05-31_Chris.csv','w', newline='', encoding='CP949')
path = r'C:\Users\DT\Desktop\자료\chromedriver'
writing = csv.writer(cover)
writing.writerow(['헤브론스타국가코드', '현지언어국가명','영문국가명','시간','대륙','GDP','인구','지역','기업식별코드','현지언어기업명','영문기업명','현지언어한줄소개내용','영문한줄소개내용','현지언어기업소개내용','영문기업소개내용','설립일자','법인등록번호','사업자등록번호','기업대표전화번호','대표팩스번호','대표이메일','기업홈페이지URL','페이스북URL','인스타그램URL','유튜브URL','링크드인URL','트위터핸들','현지언어기업주소','영문기업주소','현지언어기업상세주소','영문기업상세주소','기업우편번호','기업종업원','외감법인구분','기업연수','기업상태','현지언어담당자명','영문담당자명','현지언어직위명','영문직위명','담당자부서명','담당자전화번호','담당자팩스번호','담당자이메일','담당자이동전화번호','회계연도','유동자산금액','비유동자산금액','자산총계금액','유동부채금액','비유동부채금액','부채총계금액','자본총계금액','부채자본총계금액','매출액','매출원가금액','판매비관리비금액','영업이익손실금액','금융수익금액','금융비용금액','기타영업외수익금액','기타영업외비용금액','법인세차감전순이익','법인세비용','당기순이익','영업활동현금흐름금액','투자활동현금흐름금액','재무활동현금흐름금액','기초현금자산금액','기말현금자산금액','부채비율','영업이익율','매출액증가율','영업이익증가율','당기순이익 증가율','기업 CAGR','현지언어산업군명','영문산업군명','현지언어주요제품명내용','영문주요제품명내용','국가언어코드','현지언어언어명','영문언어명','주식시장코드','현지언어주식시장명','영문주식시장명','상장코드','상장일자','주가(일)','주가(1주)','주가(1개월)','주가(6개월)','주가(1년)','주가(3년)','주가(5년)','주가(10년)','거래량','시가총액','지점코드','지점명','주소','주소상세','우편번호','사업자등록번호','이벤트','통화구분코드','화폐단위명','담당자','소스','날짜'])

791

## This is the main code that crawls data rom SET. 
#### - Time sleep was necessary because of comparably long loading time.
#### - Uncollectable or unavailable data was displayed as null.
#### - (계산예정) Data Will be calculated and inserted manually later.
#### - Sample Visualization is located in the bottom.

In [39]:
#SET
failList = []
company_links = []
company_links2 = []
all_market_maintable = []
all_market_maintable2 = []
driver = webdriver.Chrome(path)
driver.get('https://classic.set.or.th/set/commonslookup.do?language=en&country=US')
base_url = "https://classic.set.or.th/"
count = 0

#Get each company's link
time.sleep(2)
text = driver.page_source
soup = bs4.BeautifulSoup(text,'html.parser')
maintable = soup.find('div', {"class": "col-xs-12 padding-top-10 text-center capital-letter"})
all_atag_maintable = maintable.find_all('a')

#Final Processing
for a in all_atag_maintable:
    company_link = a.attrs["href"]
    company_links.append(company_link)
    
# Main Data Collection code
for sub in company_links:
        count = count + 1
        driver.get(base_url + sub)
        time.sleep(2)
        company_links2 = []
        text2 = driver.page_source
        soup2 = bs4.BeautifulSoup(text2,'html.parser')
        maintable2 = soup2.find('table', {"class": "table table-profile table-hover table-set-border-yellow"})
        all_atag_maintable2 = maintable2.find_all('a')
        for a in all_atag_maintable2:
            company_link2 = a.attrs["href"]
            company_links2.append(company_link2)
            all_market_maintable.append(a.text)

        for sub2 in company_links2:
            try:
                driver.get(base_url + sub2)
                time.sleep(2)
                info = []
                #국가코드
                info.append("THA")
                info.append("Ratcha-anachak Thai")
                info.append("Thailand")
                info.append("GMT+07:00")
                info.append("아시아")
                info.append("522000000000 USD")
                info.append("66171439")
                info.append("동남아시아")

                #기업
                name = driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[1]/div[2]').text
                try: 
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[3]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[2]/div[3]/div[2]').text + "THA SET" + name)
                except:
                    info.append("THA SET" + name)
                info.append(name)
                info.append(name)

                #기업 + 기업연락처 + 기업주소 + 기업추가정보 + 담당자정보
                try: 
                    address1 = driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[2]/div[2]').text
                    address2 = driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[2]/div[2]').text
                except: 
                    address1 = "Thailand"
                    address2 = "Thailand, Asia"
                try:
                    info.append(name + "(English: " + name + ")" + " is a public company that is listed on SET(Stock Exchange of Thailand). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[3]/div[2]').text + ".")
                    info.append(name + "(English: " + name + ")" + " is a public company that is listed on SET(Stock Exchange of Thailand). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[3]/div[2]').text + ".")
                    info.append(name + "(English: " + name + ")" + " is a public company that is listed on SET(Stock Exchange of Thailand). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[3]/div[2]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[5]/div[2]').text)
                    info.append(name + "(English: " + name + ")" + " is a public company that is listed on SET(Stock Exchange of Thailand). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[3]/div[2]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[5]/div[2]').text)
                except:
                    info.append(name + "(English: " + name + ")" + " is a public company that is listed on SET(Stock Exchange of Thailand). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[2]/div[2]').text + ".")
                    info.append(name + "(English: " + name + ")" + " is a public company that is listed on SET(Stock Exchange of Thailand). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[2]/div[2]').text + ".")
                    info.append(name + "(English: " + name + ")" + " is a public company that is listed on SET(Stock Exchange of Thailand). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[2]/div[2]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[5]/div[2]').text)
                    info.append(name + "(English: " + name + ")" + " is a public company that is listed on SET(Stock Exchange of Thailand). Located at " + address2 + ", they are providing services related to  " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[2]/div[2]').text + ". You can find more information on company's website, " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[5]/div[2]').text)
                info.append("")
                info.append("")
                info.append("")
                try :
                    info.append("+66 " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[3]/div[2]').text) 
                except:
                    info.append("")
                try: 
                    info.append("+66 " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[4]/div[2]').text)
                except:
                    info.append("")
                info.append("")
                try:
                    url = driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[5]/div[2]')
                    if url != "-" and url != "" :
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[5]/div[2]').text)
                    else:
                        info.append(str(base_url + sub2))
                except:
                    info.append(str(base_url + sub2))
                info.append("")
                info.append("")
                info.append("")
                info.append("")
                info.append("")

                info.append(address1)
                info.append(address1)
                info.append(address2)
                info.append(address2)

                info.append("")
                info.append("")
                info.append("")
                info.append("")
                info.append("Active") 
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[6]/td/div[3]/div[1]').text)
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[6]/td/div[3]/div[1]').text)
                except:
                    info.append("")
                    info.append("")
                info.append("CHAIRMAN")
                info.append("CHAIRMAN")
                info.append("Representative")
                try :
                    info.append("+66 " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[3]/div[2]').text) 
                except:
                    info.append("")
                try: 
                    info.append("+66 " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[4]/div[2]').text)
                except:
                    info.append("")
                info.append("")
                try :
                    info.append("+66 " + driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[1]/td/div[3]/div[2]').text) 
                except:
                    info.append("")

                #재무정보
                driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[2]/div/ul/li[2]/a').click()
                time.sleep(2)
                backlink = driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/div[3]/a').get_attribute("href")
                driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/div[3]/a').click()
                time.sleep(2)   


                targetlink = driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[1]/div[2]/div[2]/a').get_attribute("href")
                driver.get(targetlink)

                #회계연도
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[1]/td[2]/strong').text)
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[1]/td[3]/strong').text)
                #유동자산금액
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[6]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[6]/td[3]').text  + "M. Baht")
                #비유동자산금액
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[8]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[8]/td[3]').text  + "M. Baht")
                #자산총계금액
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[9]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[9]/td[3]').text  + "M. Baht")
                #유동부채금액
                try: 
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[13]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[13]/td[3]').text  + "M. Baht")
                #비유동부채금액
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[14]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[14]/td[3]').text + "M. Baht")
                #부채총계금액
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[15]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[15]/td[3]').text + "M. Baht")
                #자본부채총계금액
                try:
                    info.append("M. Baht")
                except:
                    info.append("M. Baht")
                #부채자본총액
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[9]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[9]/td[3]').text + "M. Baht")
                #매출액
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[3]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[3]/td[3]').text + "M. Baht")
                #매출원가금액
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[6]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[6]/td[3]').text + "M. Baht")
                #판매비관리비금액
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[7]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[7]/td[3]').text + "M. Baht")
                #영업이익손실금액
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[12]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[12]/td[3]').text + "M. Baht")

                #금융수익금액
                try:
                    info.append("M. Baht")
                except:
                    info.append("M. Baht")
                #금융비용금액
                try:
                    info.append("M. Baht")
                except:
                    info.append("M. Baht")

                #기타영업외수익금액
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[4]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[4]/td[3]').text + "M. Baht")
                #기타영업외비용금액
                try:
                    info.append("")
                except:
                    info.append("")
                #법인세차감전순이익
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[9]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[9]/td[3]').text + "M. Baht")
                #법인세비용
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[10]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[10]/td[3]').text + "M. Baht")
                #당기순이익
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[11]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[9]/td/table/tbody/tr[11]/td[3]').text + "M. Baht")

                #현금흐름
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[10]/td/table/tbody/tr[3]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[10]/td/table/tbody/tr[3]/td[3]').text + "M. Baht")
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[10]/td/table/tbody/tr[4]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[10]/td/table/tbody/tr[4]/td[3]').text + "M. Baht")
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[10]/td/table/tbody/tr[5]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[10]/td/table/tbody/tr[5]/td[3]').text + "M. Baht")
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[3]/td[4]').text + "M. Baht")
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[3]/td[2]').text + "M. Baht")
                except:
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[3]/td[5]').text + "M. Baht")
                    info.append(driver.find_element(By.XPATH, value='/html/body/table/tbody/tr[3]/td/table[2]/tbody/tr[8]/td/table/tbody/tr[3]/td[4]').text + "M. Baht")

                info.append("")

                #재무정보중 계산예정            
                info.append("")
                info.append("")
                info.append("")
                info.append("")
                info.append("")

                driver.get(backlink)
                time.sleep(2)

                #산업군
                driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[2]/div/ul/li[1]/a').click()
                time.sleep(2)
                try: 
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[2]/div[2]').text)
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[2]/div[2]').text)
                except:
                    info.append("Public Company")
                    info.append("Public Company")
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[3]/div[2]').text)
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[3]/div[2]').text)
                except:
                    try:
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[2]/div[2]').text)
                        info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[3]/td/div/div[1]/div[2]/div[2]').text)
                    except:
                        info.append("Product and Services")
                        info.append("Product and Services")


                #언어
                info.append("THA")
                info.append("Phasa Thai")
                info.append("Thai")

                #주식시장정보
                driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[2]/div/ul/li[7]/a').click()
                time.sleep(2)
                info.append("SET")
                info.append("Stock Exchange of Thailand(SET)")
                info.append("Stock Exchange of Thailand(SET)")
                info.append(str(all_market_maintable.pop(0)))
                info.append("")
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[4]/table/tbody/tr[1]/td[3]').text + " THB")
                except:
                    info.append("")
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[4]/table/tbody/tr[7]/td[5]').text + " THB")
                except:
                    info.append("")
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[4]/table/tbody/tr[20]/td[5]').text + " THB")
                except:
                    info.append("")
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[4]/table/tbody/tr[34]/td[5]').text + " THB")
                except:
                    info.append("")
                info.append("")
                info.append("")
                info.append("")
                info.append("")

                driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div/div/div[2]/div/ul/li[6]/a').click()
                try:
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div/div/div[3]/div[1]/table[1]/tbody/tr[9]/td[2]').text)
                except:
                    info.append("")
                try:
                    driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[2]/div/ul/li[1]/a').click()
                    time.sleep(2)
                    info.append(driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[3]/table/tbody/tr[2]/td/div[3]/div[2]/div[2]').text + "M Baht")
                except:
                    info.append("")

                #지점
                info.append("")
                info.append("")
                info.append("Thailand")
                info.append("Thailand, Asia")
                info.append("")
                info.append("")

                #이벤트
                try:
                    driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[2]/div/ul/li[5]/a').click()
                    element = driver.find_element(By.XPATH, value='/html/body/div[4]/div/div[2]/div[3]/div[1]/div/div[6]/table/tbody/tr[1]/td[5]/a')
                    href = base_url + element.get_attribute('href')
                    info.append(href)
                except:
                    info.append("")

                #화폐
                info.append("THB")
                info.append("Thai Baht")

                #관리
                info.append("Chris")
                info.append("SET")
                info.append("2022-06-01")

                #Keep data in csv
                try: 
                    writing.writerow(info)
                except:
                    failList.append(info)
            except:
                failList.append(base_url + sub2)
#Result
print("-----------------------------------------")
print("# of Companies: " + count)
print("Failed Company #: " + len(failList))
print("Fail Ratio" + (len(failList) / count))
print("Failed Company Detail: ")
print(failList)
print("-----------------------------------------")

C:\Users\DT\AppData\Local\Temp\ipykernel_12884\2164517857.py:7: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)


-----------------------------------------


TypeError: can only concatenate str (not "int") to str

In [ ]:
print("Finished")

## Visualization

In [10]:
file = open("Thailand_2022_SET_2022-05-31_Chris.csv")
csvreader = csv.reader(file)
header = next(csvreader)
print(header)

StopIteration: 

In [15]:
with open("Thailand_2022_SET_2022-05-31_Chris.csv", "r") as table:
    for row in table:
        print(row)

헤브론스타국가코드,현지언어국가명,영문국가명,시간,대륙,GDP,인구,지역,기업식별코드,현지언어기업명,영문기업명,현지언어한줄소개내용,영문한줄소개내용,현지언어기업소개내용,영문기업소개내용,설립일자,법인등록번호,사업자등록번호,기업대표전화번호,대표팩스번호,대표이메일,기업홈페이지URL,페이스북URL,인스타그램URL,유튜브URL,링크드인URL,트위터핸들,현지언어기업주소,영문기업주소,현지언어기업상세주소,영문기업상세주소,기업우편번호,기업종업원,외감법인구분,기업연수,기업상태,현지언어담당자명,영문담당자명,현지언어직위명,영문직위명,담당자부서명,담당자전화번호,담당자팩스번호,담당자이메일,담당자이동전화번호,회계연도,유동자산금액,비유동자산금액,자산총계금액,유동부채금액,비유동부채금액,부채총계금액,자본총계금액,부채자본총계금액,매출액,매출원가금액,판매비관리비금액,영업이익손실금액,금융수익금액,금융비용금액,기타영업외수익금액,기타영업외비용금액,법인세차감전순이익,법인세비용,당기순이익,영업활동현금흐름금액,투자활동현금흐름금액,재무활동현금흐름금액,기초현금자산금액,기말현금자산금액,부채비율,영업이익율,매출액증가율,영업이익증가율,당기순이익 증가율,기업 CAGR,현지언어산업군명,영문산업군명,현지언어주요제품명내용,영문주요제품명내용,국가언어코드,현지언어언어명,영문언어명,주식시장코드,현지언어주식시장명,영문주식시장명,상장코드,상장일자,주가(일),주가(1주),주가(1개월),주가(6개월),주가(1년),주가(3년),주가(5년),주가(10년),거래량,시가총액,지점코드,지점명,주소,주소상세,우편번호,사업자등록번호,이벤트,통화구분코드,화폐단위명,담당자,소스,날짜



In [16]:
        targetlink = "https://classic.set.or.th/set/factsheet.do?symbol=2S&ssoPageId=3&language=en&country=US"
        res = requests.get(targetlink)
        table = pd.read_html(res.text)[24]
        table.columns = table.iloc[0]
        table[table['Statement of Financial Position (MB.)']=='Cash And Cash Equivalents']

,Statement of Financial Position (MB.),3M/2022 (01/01/22 -31/03/22),3M/2021 (01/01/21 -31/03/21),2021 (01/01/21 -31/12/21),2020 (01/01/20 -31/12/20),2019 (01/01/19 -31/12/19)
2,Cash And Cash Equivalents,610.04,106.52,352.59,132.11,131.85
